### import libs

In [43]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
from torch.autograd import Variable
from torchvision import datasets, transforms

### basic autograd example 1

In [16]:
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)

In [17]:
y = w*x + b
y.backward()

print(x.grad, w.grad, b.grad)

tensor([ 2.]) tensor([ 1.]) tensor([ 1.])


### basic autograd example 2

In [18]:
x = Variable(torch.randn(5,3))
y = Variable(torch.randn(5,2))

In [21]:
linear = nn.Linear(3,2)

In [22]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [34]:
pred = linear(x)

loss = criterion(pred, y)
print('loss: ', loss.data[0])

loss:  tensor(1.6917)


/home/yangshuang/.conda/envs/yangshuang/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


In [24]:
loss.backward()

In [26]:
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[ 0.0834, -0.0962,  0.3447],
        [ 0.7191, -0.1140, -0.3563]])
dL/db:  tensor([-0.4584,  0.5111])


In [32]:
optimizer.step()

In [33]:
pred = linear(x)
loss = criterion(pred, y)
loss.data

tensor(1.6917)

### loading data from numpy

In [37]:
a = np.array([[1,2], [3,4]])
b = torch.from_numpy(a) # numpy to torch tensor
c = b.numpy() # torch tensor to numpy

### input pipline

In [44]:
train_dataset = datasets.CIFAR10(root='../data/',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)

In [45]:
image, label = train_dataset[0]
print(image.size(), label)

torch.Size([3, 32, 32]) 6


In [48]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

In [54]:
data_iter = iter(train_loader)
image, labels = data_iter.next() # one batch

In [55]:
for images, labels in train_loader:
    pass

### input pipline for custorm dataset

In [56]:
class MyDataset(data.Dataset):
    def __init__(self):
        # init file path pr file names
        pass
    
    def __getiterm__(self, index):
        # read data, preprocess data, return image and label
        pass
    
    def __len__(self):
        return 0
    
my_dataset = MyDataset()
train_loader = torch.utils.data.DataLoader(dataset=my_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

### using pretraned model

In [65]:
resnet = torchvision.models.resnet34(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Linear(resnet.fc.in_features, 100)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/yangshuang/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:03<00:00, 27647849.63it/s]


In [66]:
images = Variable(torch.randn(10, 3, 224, 224))
output = resnet(images)
print(output.size())

torch.Size([10, 100])


### save and load the model

In [67]:
# save and load the entire model
torch.save(resnet, 'model.pkl')
model = torch.load('model.pkl')

In [68]:
# save and load the model parameters(recommended)
torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))